In [11]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import statsmodels.api as sm
from timeit import timeit
from numba import double
from numba import jit
from joblib import Parallel, delayed


In [12]:
reps, beta, n_array = 1000, [2,3,4], [10, 100, 1000, 10000, 100000, 10000000]

In [13]:
def python_boot(arg_reps,arg_row_id,arg_n,arg_X,arg_Y):
    store_beta = np.zeros((reps,X.shape[1]))
    for r in range(arg_reps):
        this_sample = np.random.choice(arg_row_id, size=arg_n, replace=True) # gives sampled row numbers
        # Define data for this replicate:    
        X_r = arg_X[this_sample,:]
        Y_r = arg_Y[this_sample]
        # Estimate model 
        store_beta[r,:] = sm.regression.linear_model.OLS(Y_r,X_r).fit(disp=0).params
    return store_beta


In [14]:
@jit(nogil=True)
def python_boot_numba(arg_reps,arg_row_id,arg_n,arg_X,arg_Y):
    store_beta = np.zeros((1000,3))
    for r in range(arg_reps):
        this_sample = np.random.choice(arg_row_id, size=arg_n, replace=True) # gives sampled row numbers
        # Define data for this replicate:    
        X_r = arg_X[this_sample,:]
        Y_r = arg_Y[this_sample]
        # Estimate model 
        store_beta[r,:] = sm.regression.linear_model.OLS(Y_r,X_r).fit(disp=0).params
    return store_beta

In [15]:

for n in n_array:
    row_id = range(0,n)
    X1 = np.random.normal(10,4,(n,1))
    X2 = np.random.normal(10,4,(n,1))
    X=np.append(X1,X2,1)
    X = np.append(X,np.tile(1,(n,1)),1)
    error = np.random.randn(n,1)
    Y = np.dot(X,beta).reshape((n,1)) + error
    print ("Number of observations= ",n)
    %timeit python_boot(reps,row_id,n,X,Y)
    %timeit python_boot_numba(reps,row_id,n,X,Y)

Number of observations=  10
175 ms ± 2.01 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
177 ms ± 3.39 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Number of observations=  100
The slowest run took 4.85 times longer than the fastest. This could mean that an intermediate result is being cached.
433 ms ± 284 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
227 ms ± 7.15 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Number of observations=  1000
418 ms ± 22.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
432 ms ± 24.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Number of observations=  10000


KeyboardInterrupt: 

Copy pasted results for future use.


Number of observations=  10
166 ms ± 381 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
170 ms ± 376 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Number of observations=  100
216 ms ± 5.68 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
219 ms ± 9.46 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Number of observations=  1000
405 ms ± 9.39 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
396 ms ± 11.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Number of observations=  10000
2.17 s ± 65.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.2 s ± 52 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Number of observations=  100000
23.8 s ± 335 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
23.1 s ± 169 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Number of observations=  1000000
4min 40s ± 15.1 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


Number of observations=  10
44.8 ms ± 1.78 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Number of observations=  100
53.7 ms ± 2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Number of observations=  1000
155 ms ± 5.54 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Number of observations=  10000
1.22 s ± 52.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Number of observations=  100000
14.3 s ± 84.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Copy pasted results from matlab for future use.

10	3.03084240000000
100	3.11670380000000
1000	3.32423630000000
10000	4.80083280000000
100000	19.6583377000000
1000000	191.762040900000


10	0.0382322000000000
100	0.0414775000000000
1000	0.0914098000000000
10000	0.658673000000000
100000	8.05412470000000
1000000	124.202653400000

In [ ]:
matlab_data1=pd.DataFrame([[10,3.03],[100, 3.11], [1000,3.324], [10000, 4.8008], [100000, 19.658], [1000000,191.762]],columns=['n','Matlab Time'])
matlab_data2=pd.DataFrame([[10,.038],[100, .0415], [1000,.0914], [10000, .6586], [100000, 8.054], [1000000, 124.2]],columns=['n','Matlab Time Linear Algebra'])
python_data1 = pd.DataFrame([[10,0.166],[100, 0.216], [1000,0.405], [10000, 2.17], [100000, 23.8], [1000000,280]],columns=['n','Python Time'])
python_data2 = pd.DataFrame([[10,0.170],[100, 0.219], [1000,0.396], [10000, 2.2], [100000, 23.1], [1000000,266]],columns=['n','Python JIT Time'])
python_data3 = pd.DataFrame([[10,0.044],[100, 0.053], [1000,0.155], [10000, 1.22], [100000, 14.3], [1000000,211]],columns=['n','Python Time Linear Algebra'])
plot_data = pd.concat([matlab_data1, matlab_data2['Matlab Time Linear Algebra'],python_data1['Python Time'],python_data2['Python JIT Time'],python_data3['Python Time Linear Algebra']], axis=1)
plot_data = plot_data.set_index('n')
print(plot_data)
plot_data.plot(kind = "bar")

plot_data = plot_data.drop([1000000])
plot_data.plot(kind = "bar")

plot_data = plot_data.drop([100000])
plot_data.plot(kind = "bar")

plot_data = plot_data.drop([10000])
plot_data.plot(kind = "bar")

